In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Rutas (ajusta si tus archivos están en otra carpeta)
PATH_STORES = Path("bookshop-stores-sales.txt")
PATH_WEB    = Path("bookshop-web-sales.txt")
OUTPUT_XLSX = Path("bookshop_sales_report.xlsx")


In [2]:
def load_and_clean_stores_txt(path: Path) -> pd.DataFrame:
    # Leer como CSV genérico (sin encabezado)
    df = pd.read_csv(
        path,
        header=None,
        names=["Title", "Units sold", "List price", "Royalty"],
        dtype=str,  # leemos todo como string para limpiar con control
        keep_default_na=True
    )

    # Quitar filas completamente vacías (todas NaN)
    df = df.dropna(how="all").copy()

    # Quitar filas con "Sales report for Q4" en Title
    df = df[df["Title"].fillna("").str.strip() != "Sales report for Q4"].copy()

    # Crear columna auxiliar para marcar el bloque de tienda
    # Detecta las líneas "Bookshop US Store", "Bookshop UK Store", "Bookshop France Store"
    df["Store_block"] = np.where(
        df["Title"].fillna("").str.contains(r"^Bookshop (US|UK|France) Store$", regex=True),
        df["Title"].str.strip(),
        np.nan
    )

    # Propagar el bloque hacia abajo (forward-fill)
    df["Store_block"] = df["Store_block"].ffill()

    # Eliminar filas que son encabezados de bloque o totales de regalías
    mask_header = df["Title"].fillna("").str.match(r"^Bookshop (US|UK|France) Store$")
    mask_totals = df["Title"].fillna("").str.contains(r"royalties", case=False)
    df = df[~(mask_header | mask_totals)].copy()

    # Convertir columnas numéricas (si no son NaN)
    for col in ["Units sold", "List price", "Royalty"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Quitar filas con 3 o más NaN (por seguridad)
    df = df[df.isna().sum(axis=1) < 3].copy()

    # Mapear Store según bloque (según tu requerimiento)
    # US -> "US"
    # UK -> "EUR"
    # France -> "EUR"
    def map_store(block: str) -> str:
        if pd.isna(block):
            return np.nan
        if "US Store" in block:
            return "US"
        if "UK Store" in block:
            return "EUR"
        if "France Store" in block:
            return "EUR"
        return np.nan

    df["Store"] = df["Store_block"].apply(map_store)

    # Reordenar columnas y limpiar espacios en Title
    df["Title"] = df["Title"].str.strip()
    df = df[["Store", "Title", "Units sold", "List price", "Royalty"]].copy()

    return df


In [3]:
def load_and_clean_web_txt(path: Path) -> pd.DataFrame:
    dfw = pd.read_csv(
        path,
        header=None,
        names=["Store", "Title", "Units sold", "List price", "Royalty"],
        dtype=str,
        keep_default_na=True
    )

    # Si el archivo web ya trae "Store" en la primera columna, la respetamos;
    # si no, la creamos como "WEB".
    if "Store" not in dfw.columns or dfw["Store"].isna().all():
        dfw["Store"] = "WEB"
    else:
        # Normalizamos a "WEB" por requerimiento
        dfw["Store"] = "WEB"

    # Quitar filas completamente vacías
    dfw = dfw.dropna(how="all").copy()

    # Quitar filas con "Sales report for Q4" en Title (por si existiera)
    dfw = dfw[dfw["Title"].fillna("").str.strip() != "Sales report for Q4"].copy()

    # Convertir columnas numéricas
    for col in ["Units sold", "List price", "Royalty"]:
        dfw[col] = pd.to_numeric(dfw[col], errors="coerce")

    # Quitar filas con 3 o más NaN (por seguridad)
    dfw = dfw[dfw.isna().sum(axis=1) < 3].copy()

    # Limpiar espacios y reordenar
    dfw["Title"] = dfw["Title"].str.strip()
    dfw = dfw[["Store", "Title", "Units sold", "List price", "Royalty"]].copy()

    return dfw


In [4]:
df_stores = load_and_clean_stores_txt(PATH_STORES)
df_web    = load_and_clean_web_txt(PATH_WEB)

# Unir ambos (detalle de tiendas físicas + web)
df_all = pd.concat([df_stores, df_web], ignore_index=True)

# Orden opcional por Store y Title
df_all = df_all.sort_values(["Store", "Title"], kind="stable").reset_index(drop=True)

df_all


/tmp/ipykernel_26878/411000355.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["Title"].fillna("").str.contains(r"^Bookshop (US|UK|France) Store$", regex=True),


,Store,Title,Units sold,List price,Royalty
0,EUR,Pining for the Fisheries of Yore,47.00,2.99,11.98
1,EUR,Swimrand,8.00,1.99,0.88
2,EUR,The Bricklayer's Bible,17.00,2.99,3.50
3,EUR,The Duck Goes Here,12.00,1.99,1.50
4,EUR,The Tower Commission Report,4.00,6.50,4.80
5,US,Pining for the Fisheries of Yore,80.00,3.50,14.98
6,US,Swimrand,1.00,2.99,0.14
7,US,The Bricklayer's Bible,17.00,3.50,5.15
8,US,The Duck Goes Here,34.00,2.99,5.78
9,US,The Tower Commission Report,4.00,9.50,6.20
